# NYC 311 Service Requests Data Pipeline

*Library + jobs to fetch, cache, and analyze NYC 311 service request data.*

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# As Jobs

## LTP Pipeline

In [ ]:
from pathlib import Path

# Per-user, cross-platform base under the home dir
# retrieve for one day only:
dir_base = Path.home() / "TASK_HBC_TSY"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base={dir_base} \
  --incremental=True \
  --log-level=INFO
%cd -q notebooks/

In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=5
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

### Imports

In [2]:
import sys
from pathlib import Path

p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [3]:
from hbc import app_context, DataContainer, utils as ul

In [4]:
from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [5]:
app_context

AppContext
as_of : 2025-12-24
dir_analytics: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS'),
dir_base: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp'),
dir_cache: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/CACHE'),
dir_logging: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS')

## Logging

In [6]:
import logging
logger = logging.getLogger()

In [7]:
# Console-only (no file writes):
ul.conf_log(level=logging.DEBUG, console=True, file=False, reset_handlers=True)

# File-only (no console output at all):
ul.conf_log(level=logging.INFO, console=False, file=True, reset_handlers=True)

# Both:
ul.conf_log(level=logging.DEBUG, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt
Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (DEBUG)>

In [8]:
logger.debug('message')

2025-12-24 20:55:33 1324733693.py           1 DEBUG root    : message


## DataContainer

In [ ]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [ ]:
# we retrieve 100 rows
dc.get()

In [ ]:
dc.get(where=f"created_date = '{ul.date_as_iso_format(ul.str_as_date('20091231'))}' ")

In [ ]:
dc.to_cache()

In [ ]:
dc.from_cache().head()

In [ ]:
', '.join(dc.all_cached_dates)

## Analytics

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df = df[~df[cols.DROP_FLAG]]

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(ul.get_dir_analytics() / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

### new dataset: nyc_open_data_311_call_center_inquiry:

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"agency='NYPD'", limit=250)

In [ ]:
dc.df.shape

In [ ]:
dc.df.head()

In [ ]:
dc.df.head()

In [ ]:
dc.get(where=f"date = '{ul.date_as_iso_format(ul.str_as_date('2010-01-03'))}'")

In [ ]:
dc.df.head()

### new_dataset: nyc_open_data_311_customer_satisfaction_survey

In [11]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [12]:
dc.get()

2025-12-24 20:55:45 connectionpool.py    1049 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): data.cityofnewyork.us:443
2025-12-24 20:55:46 connectionpool.py     544 DEBUG urllib3.connectionpool: https://data.cityofnewyork.us:443 "GET /resource/kizp-4dfk.json?%24limit=100 HTTP/1.1" 200 None
2025-12-24 20:55:46 fetch_nycopen.py       71 INFO  root    : Fetched 100 rows
2025-12-24 20:55:46 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-24 20:55:46 base.py                53 INFO  root    : cleaning...
2025-12-24 20:55:46 base.py                56 INFO  root    : normalizing...
2025-12-24 20:55:46 base.py                59 INFO  root    : validating...
2025-12-24 20:55:46 base.py                62 INFO  root    : finalizing...


In [ ]:
dc.get(where=f"answer_satisfaction='Neutral'")

In [ ]:
dc.df.head()

# Cache To SQLLite DataBase Via Asp.Net Rest Api with EF Core

In [65]:
import hbc
from hbc import DataContainer
from hbc.ltp.persistence.db import SqlLiteDataBase

In [66]:
logger.setLevel(logging.INFO)

In [70]:
import os
from pathlib import Path

# Adjust the parent() call if your notebook lives deeper/shallower.
repo_root = Path.cwd().parents[1]  # e.g., notebooks -> hbc_py -> repo root
os.environ["HBC_DB_PATH"] = str(repo_root / "hbc_db" / "hbc.db")
os.environ["HBC_API_URL"] = "http://localhost:5047"



dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")
db = SqlLiteDataBase()

dc.get(limit=400)

dc.to_cache()

dc.from_cache(query=f'select * from {dc.moniker}').shape

2025-12-24 21:30:47 fetch_nycopen.py       71 INFO  root    : Fetched 400 rows
2025-12-24 21:30:47 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-24 21:30:47 base.py                53 INFO  root    : cleaning...
2025-12-24 21:30:47 base.py                56 INFO  root    : normalizing...
2025-12-24 21:30:47 base.py                59 INFO  root    : validating...
2025-12-24 21:30:47 base.py                62 INFO  root    : finalizing...
2025-12-24 21:30:47 db.py                 205 INFO  root    : Posting batch 1-100/400 to http://localhost:5047/surveys/batch (verify=False)


ConnectionError: HTTPConnectionPool(host='localhost', port=5047): Max retries exceeded with url: /surveys/batch (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15985df20>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
dc